In [1]:
import numpy as np
import sys
import os
from os import listdir
import re
import pandas as pd
import glob
import unicodedata
import string

import copy
import pickle

from collections import Counter

import tensorflow as tf

In [2]:
path = "./data/finished_files/chunked/" # test_000.bin"
train_raw_all = []

for filename in listdir(path):
    if 'train_00' in filename:
        print(filename)
        path_file = path + '\\' + filename
        with open(path_file, 'r', encoding='utf-8',errors='ignore') as binary_file:
            raw_text = binary_file.read()
            #raw_text = raw_text.replace('<s>', '')
            #raw_text = raw_text.replace('</s>', '')
            raw_text = raw_text.replace('\n', '')
            raw_text = raw_text.replace('\x12', '')
            raw_text = raw_text.replace('\x17', '')
            raw_text = raw_text.replace('\x01', '')
            raw_text = raw_text.replace('\x02', '')
            raw_text = raw_text.replace('\x03', '')
            raw_text = raw_text.replace('\x00', '')
            raw_text = raw_text.replace('\x11', '')
            raw_text = raw_text.replace('\x14', '')
            raw_text = raw_text.replace('\x15', '')
            abstract = raw_text.split('\x08abstract') # .decode('utf-8', 'ignore')ISO-8859-1 
            x = [a.split('\x07article') for a in abstract]
            train_raw_all.extend(x[1:])
   
        
# df = pd.DataFrame(data=train_raw_all, columns=['abstract','article']) # df = df.ix[1:]
# s = {'<s>':'', '</s>':''}
# df['abstract'] = df['abstract'].replace(s)

#df['abstract'] = df['abstract'].apply(lambda x: x.split('<s>',1)[-1])
#df['article'] = df['article'].apply(lambda x: x.split('-rrb-',1)[-1])

print('done!')

train_000.bin
train_001.bin
train_002.bin
train_003.bin
train_004.bin
train_005.bin
train_006.bin
train_007.bin
train_008.bin
train_009.bin
done!


In [3]:
(train_raw_all[737])

["<s> lubos michel will referee wednesday 's champions league final in moscow . </s> <s> fellow-slovakians roman slysko and martin balko will be his two assistants . </s> <s> vladimir hrinak will be fourth official at the manchester utd v chelsea clash . </s>",
 "moscow , russia -- uefa have confirmed that slovakian referee lubos michel will take charge of wednesday 's champions league final between manchester united and chelsea in moscow . michel refereed the 2003 uefa cup final when jose mourinho 's porto beat celtic . michel , 40 last week , is remembered by chelsea fans for controversially awarding a goal to liverpool against the londoners in the semifinals of the 2005 competition . chelsea 's manager at the time jose mourinho always insisted that luis garcia 's shot did not cross the line . michel is regarded as one of the top referees in the world and officiated at euro 2004 and the 2006 world cup , including the tense germany v argentina game in the first knock-out round . he al

In [3]:
source_text = (' <END> ').join([a[1] for a in train_raw_all])
target_text = (' <END> ').join([a[0] for a in train_raw_all])

(target_text[:1111])

"<s> mentally ill inmates in miami are housed on the `` forgotten floor '' </s> <s> judge steven leifman says most are there as a result of `` avoidable felonies '' </s> <s> while cnn tours facility , patient shouts : `` i am the son of the president '' </s> <s> leifman says the system is unjust and he 's fighting for change . </s>  <END> <s> harry potter star daniel radcliffe gets # 20m fortune as he turns 18 monday . </s> <s> young actor says he has no plans to fritter his cash away . </s> <s> radcliffe 's earnings from first five potter films have been held in trust fund . </s> <END> <s> new : `` i thought i was going to die , '' driver says . </s> <s> man says pickup truck was folded in half ; he just has cut on face . </s> <s> driver : `` i probably had a 30 - , 35-foot free fall '' </s> <s> minnesota bridge collapsed during rush hour wednesday . </s>  <END> <s> parents beam with pride , ca n't stop from smiling from outpouring of support . </s> <s> mom : `` i was so happy i did n

In [156]:
# df = df[['abstract', 'article']]
# df['abstract'] = df['abstract'].apply(lambda x: x.strip(' '))

# source_text = (df['article'])
# target_text = str(df['abstract'].apply(lambda x: str(x)))

In [21]:
print('Total no. of abstracts/articles: {}'.format(len(source_text.split('<END>'))))

Total no. of abstracts/articles: 10000


In [22]:
print('Approximate number of unique words in all articles: {}'.format(len({word:None for word in str(source_text).split()})))
print('Approximate number of unique words in all abtracts: {}'.format(len({word:None for word in str(target_text).split()})))

Approximate number of unique words in all articles: 103110
Approximate number of unique words in all abtracts: 30121


In [23]:
arti = source_text.split(' <END> ')
abst = target_text.split(' <END> ')

word_counts1 = [len(a.split()) for a in arti]
word_counts2 = [len(a.split()) for a in abst]

print('Average number of words in an article: {}'.format(np.average(word_counts1)))
print('Average number of words in an abstract: {}'.format(np.average(word_counts2)))

Average number of words in an article: 720.847
Average number of words in an abstract: 55.8814


In [5]:
# new 
def extract_character_vocab(data):
    special_words = ['<PAD>', '<UNK>', '<GO>',  ' <END> ']

    # set_words = set([word for line in data.split() for word in line])
    set_words = set(data.split())
    int_to_vocab = {word_i: word for word_i, word in enumerate(special_words + list(set_words))}
    vocab_to_int = {word: word_i for word_i, word in int_to_vocab.items()}

    return int_to_vocab, vocab_to_int


# Build int2vocab and vocab2int dicts
source_int_to_vocab, source_vocab_to_int = extract_character_vocab(source_text)
target_int_to_vocab, target_vocab_to_int = extract_character_vocab(target_text)


# Convert characters to ids
source_vocab_ids = [[source_vocab_to_int.get(word, source_vocab_to_int['<UNK>']) for word in article]\
                     for article in source_text.split(' <END> ')]

target_vocab_ids = [[target_vocab_to_int.get(word, target_vocab_to_int['<UNK>']) for word in abstract]\
                     + [target_vocab_to_int[' <END> ']] for abstract in target_text.split(' <END> ')] 


## CHECKPOINT 1 

In [4]:
"""
CHECK TENSORFLOW & GPU
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from tensorflow.python.layers.core import Dense

# Check TensorFlow Version
# assert LooseVersion(tf.__version__) in [LooseVersion('1.0.0'), LooseVersion('1.0.1')], 'This project requires TensorFlow version 1.0  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0


C:\Users\Jermyn Bek\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: UserWarning: No GPU found. Please use a GPU to train your neural network.
  from ipykernel import kernelapp as app


In [8]:
# import problem_unittests

def load_preprocess():
    """
    Load the Preprocessed Training data and return them in batches of <batch_size> or less
    """
    with open('preprocess.p', mode='rb') as in_file:
        return pickle.load(in_file)


(source_vocab_ids, target_vocab_ids), (source_vocab_to_int, target_vocab_to_int), (source_int_to_vocab, target_int_to_vocab) = load_preprocess()

In [15]:
len(source_vocab_to_int)

103114

### Hyperparameters


In [16]:
# Number of Epochs
epochs = 60

# Batch Size
batch_size = 128

# RNN Size
rnn_size = 50

# Number of Layers
num_layers = 2

# Embedding Size
encoding_embedding_size = 155
decoding_embedding_size = 155

# Learning Rate
learning_rate = 0.001

### model_inputs()
To create TF Placeholders for the Neural Network:

In [17]:
"""
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None,None], name='input')
    targets = tf.placeholder(tf.int32, [None,None], name='target')
    learning_rates = tf.placeholder(tf.float32)
    keep_probs = tf.placeholder(tf.float32, name='keep_prob')
    
    return inputs, targets, learning_rates, keep_probs
"""
# new
def get_model_inputs():
    input_data = tf.placeholder(tf.int32, [None, None], name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')
    lr = tf.placeholder(tf.float32, name='learning_rate')

    target_sequence_length = tf.placeholder(tf.int32, (None,), name='target_sequence_length')
    max_target_sequence_length = tf.reduce_max(target_sequence_length, name='max_target_len')
    source_sequence_length = tf.placeholder(tf.int32, (None,), name='source_sequence_length')
    
    return input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length

# problem_unittests.test_model_inputs(model_inputs)

### encoding_layer() 
To create an encoder RNN

In [18]:
"""
def encoding_layer(rnn_inputs, rnn_size, num_layers, keep_prob):
    # tf v1.0
    # LSTM = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    # enc_cell = tf.contrib.rnn.MultiRNNCell([LSTM] * num_layers)
    
    def lstm_cell():
        return tf.nn.rnn_cell.BasicLSTMCell(rnn_size)
    
    stacked_lstm = tf.nn.rnn_cell.MultiRNNCell([lstm_cell() for _ in range(num_layers)])
    stacked_lstm2 = tf.nn.rnn_cell.DropoutWrapper(stacked_lstm, output_keep_prob=keep_prob)
    enc_output, enc_state = tf.nn.dynamic_rnn(stacked_lstm2, rnn_inputs, dtype=tf.float32)
    
    return enc_state
"""

### new
def encoding_layer(input_data, rnn_size, num_layers, source_sequence_length, 
                   source_vocab_size, encoding_embedding_size):
    
    # Encoder embedding
    enc_embed_input = tf.contrib.layers.embed_sequence(input_data, source_vocab_size, encoding_embedding_size)

    # RNN cell
    def make_cell(rnn_size):
        enc_cell = tf.contrib.rnn.LSTMCell(rnn_size,
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return enc_cell

    enc_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
    
    enc_output, enc_state = tf.nn.dynamic_rnn(enc_cell, enc_embed_input, sequence_length=source_sequence_length, dtype=tf.float32)
    
    return enc_output, enc_state



# DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE

# problem_unittests.test_encoding_layer(encoding_layer) 
# pip install dask --upgrade

### process_decoding_input() 

To remove the last word id from each batch in target_data and concat the GO ID to the beginning of each batch

In [19]:
"""
def process_decoding_input(target_data, target_vocab_to_int, batch_size):
    ending = tf.strided_slice(target_data, begin=[0,0], end=[batch_size,-1], strides=[1,1])
    decoder_input = tf.concat([tf.fill(dims=[batch_size,1], value=target_vocab_to_int['<GO>']), ending], axis=1)
    
    return decoder_input
"""

# new
def process_decoder_input(target_data, target_vocab_to_int, batch_size):
    ending = tf.strided_slice(target_data, [0, 0], [batch_size, -1], [1, 1])
    dec_input = tf.concat([tf.fill([batch_size, 1], target_vocab_to_int['<GO>']), ending], 1)

    return dec_input

# problem_unittests.test_process_decoding_input(process_decoding_input)

### decoding_layer() 


In [20]:
def decoding_layer(target_vocab_to_int, decoding_embedding_size, num_layers, rnn_size,
                   target_sequence_length, max_target_sequence_length, enc_state, dec_input):
    
    # 1. Decoder Embedding
    target_vocab_size = len(target_vocab_to_int)
    dec_embeddings = tf.Variable(tf.random_uniform([target_vocab_size, decoding_embedding_size]))
    dec_embed_input = tf.nn.embedding_lookup(params=dec_embeddings, ids=dec_input)

    # 2. Construct the decoder cell
    def make_cell(rnn_size):
        dec_cell = tf.contrib.rnn.LSTMCell(rnn_size, 
                                           initializer=tf.random_uniform_initializer(-0.1, 0.1, seed=2))
        return dec_cell

    dec_cell = tf.contrib.rnn.MultiRNNCell([make_cell(rnn_size) for _ in range(num_layers)])
     
    # 3. Dense layer to translate the decoder's output at each time 
    # step into a choice from the target vocabulary
    output_layer = Dense(target_vocab_size,
                         kernel_initializer = tf.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    # 4. Set up a training decoder and an inference decoder
    # Training Decoder
    with tf.variable_scope("decode"):

        # Helper for the training process. Used by BasicDecoder to read inputs.
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=target_sequence_length,
                                                            time_major=False)
        
        
        # Basic decoder
        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, training_helper,
                                                           enc_state, output_layer) 
        
        # Perform dynamic decoding using the decoder
        training_decoder_output = tf.contrib.seq2seq.dynamic_decode(training_decoder, impute_finished=True, 
                                                                    maximum_iterations=max_target_sequence_length)[0]
        
    # 5. Inference Decoder
    # Reuses the same parameters trained by the training process
    with tf.variable_scope("decode", reuse=True):
        start_tokens = tf.tile(tf.constant([target_vocab_to_int['<GO>']], dtype=tf.int32), [batch_size], name='start_tokens')

        # Helper for the inference process.
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(dec_embeddings, 
                                                                    start_tokens, 
                                                                    target_vocab_to_int[' <END> '])

        # Basic decoder
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell, inference_helper, enc_state, output_layer)
        
        # Perform dynamic decoding using the decoder
        inference_decoder_output = tf.contrib.seq2seq.dynamic_decode(inference_decoder, impute_finished=True, 
                                                                     maximum_iterations=max_target_sequence_length)[0]
    
    return training_decoder_output, inference_decoder_output

### Seq2seq model

merge encoder with decoder:

In [21]:
def seq2seq_model(input_data, targets, lr, target_sequence_length, 
                  max_target_sequence_length, source_sequence_length,
                  source_vocab_size, target_vocab_size,
                  encoding_embedding_size, decoding_embedding_size, 
                  rnn_size, num_layers):
    
    # Pass the input data through the ENCODER. We'll ignore the encoder output, but use the state
    _, enc_state = encoding_layer(input_data, 
                                  rnn_size, 
                                  num_layers, 
                                  source_sequence_length,
                                  source_vocab_size, 
                                  encoding_embedding_size)
    
    
    # PREPARE the target sequences we'll feed to the decoder in training mode
    dec_input = process_decoder_input(targets, target_vocab_to_int, batch_size)
    
    # Pass encoder state and decoder inputs to the decoders
    training_decoder_output, inference_decoder_output = decoding_layer(target_vocab_to_int, 
                                                                       decoding_embedding_size, 
                                                                       num_layers, 
                                                                       rnn_size,
                                                                       target_sequence_length,
                                                                       max_target_sequence_length,
                                                                       enc_state, 
                                                                       dec_input) 
    
    return training_decoder_output, inference_decoder_output

# note: both contain a 'rnn_output' logits tensor

In [22]:
# build the graph

train_graph = tf.Graph()
# Set the graph to default to ensure that it is ready for training
with train_graph.as_default():
    
    # Load the model inputs    
    input_data, targets, lr, target_sequence_length, max_target_sequence_length, source_sequence_length = get_model_inputs()
    
    # Create the training and inference logits
    training_decoder_output, inference_decoder_output = seq2seq_model(input_data, 
                                                                      targets, 
                                                                      lr, 
                                                                      target_sequence_length, 
                                                                      max_target_sequence_length, 
                                                                      source_sequence_length,
                                                                      len(source_vocab_to_int),
                                                                      len(target_vocab_to_int),
                                                                      encoding_embedding_size, 
                                                                      decoding_embedding_size, 
                                                                      rnn_size, 
                                                                      num_layers)    
    
    # Create tensors for the training logits and inference logits
    training_logits = tf.identity(training_decoder_output.rnn_output, 'logits') # ?
    inference_logits = tf.identity(inference_decoder_output.sample_id, name='predictions') # ?
    
    # Create the weights for sequence_loss
    masks = tf.sequence_mask(target_sequence_length, max_target_sequence_length, dtype=tf.float32, name='masks')

    with tf.name_scope("optimization"):
        
        # Loss function
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, targets, masks)

        # Optimizer
        optimizer = tf.train.AdamOptimizer(lr)

        # Gradient Clipping
        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

### Get batches

In [23]:
# Pad articles/abstracts with <PAD> so that each articles/abstracts of a batch has the same length

def pad_batch(article_batch, pad_int):
    max_article = max([len(a) for a in article_batch])
    
    return [a + [pad_int] * (max_article - len(a)) for a in article_batch]

In [24]:
# Batch targets, sources, and the lengths of their articles/abstracts together

def get_batches(targets, sources, batch_size, source_pad_int, target_pad_int):
    for batch_i in range(0, len(sources)//batch_size):
        start_i = batch_i * batch_size
        sources_batch = sources[start_i:start_i + batch_size]
        targets_batch = targets[start_i:start_i + batch_size]
        pad_sources_batch = np.array(pad_batch(sources_batch, source_pad_int))
        pad_targets_batch = np.array(pad_batch(targets_batch, target_pad_int))
        
        # Need the lengths for the _lengths parameters
        pad_targets_lengths = []
        for target in pad_targets_batch:
            pad_targets_lengths.append(len(target))
        
        pad_source_lengths = []
        for source in pad_sources_batch:
            pad_source_lengths.append(len(source))
        
        yield pad_targets_batch, pad_sources_batch, pad_targets_lengths, pad_source_lengths

### Train!

In [ ]:
# Split data to training and validation sets
train_source = source_vocab_ids[batch_size:]
train_target = target_vocab_ids[batch_size:]
valid_source = source_vocab_ids[:batch_size]
valid_target = target_vocab_ids[:batch_size]

(valid_targets_batch, valid_sources_batch, 
 valid_targets_lengths, valid_sources_lengths) = next(get_batches(valid_target,
                                                                  valid_source,
                                                                  batch_size,
                                                                  source_vocab_to_int['<PAD>'],
                                                                  target_vocab_to_int['<PAD>']))

display_step = 2 # Check training loss after every 20 batches

checkpoint = "best_model.ckpt" 
with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())
        
    for epoch_i in range(1, epochs+1):
        for batch_i, (targets_batch, sources_batch, targets_lengths, sources_lengths) in\
        enumerate(get_batches(train_target, train_source, batch_size,
                              source_vocab_to_int['<PAD>'],target_vocab_to_int['<PAD>'])):
            
            # Training step
            _, loss = sess.run(
                [train_op, cost],
                {input_data: sources_batch,
                 targets: targets_batch,
                 lr: learning_rate,
                 target_sequence_length: targets_lengths,
                 source_sequence_length: sources_lengths})

            # Debug message updating us on the status of the training
            if batch_i % display_step == 0 and batch_i > 0:
                
                # Calculate validation cost
                validation_loss = sess.run(
                [cost],
                {input_data: valid_sources_batch,
                 targets: valid_targets_batch,
                 lr: learning_rate,
                 target_sequence_length: valid_targets_lengths,
                 source_sequence_length: valid_sources_lengths})
                
                print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}  - Validation loss: {:>6.3f}'
                      .format(epoch_i,
                              epochs, 
                              batch_i, 
                              len(train_source) // batch_size, 
                              loss, 
                              validation_loss[0]))

    
    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, checkpoint)
    print('Model Trained and Saved')

### Prediction 

In [ ]:
# TODO...

In [ ]:
def source_to_seq(text):
    '''Prepare the text for the model'''
    sequence_length = 7
    return [source_letter_to_int.get(word, source_letter_to_int['<UNK>']) for word in text]+ [source_letter_to_int['<PAD>']]*(sequence_length-len(text))

In [ ]:
input_sentence = 'hello'
text = source_to_seq(input_sentence)

checkpoint = "./best_model.ckpt"

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(checkpoint + '.meta')
    loader.restore(sess, checkpoint)

    input_data = loaded_graph.get_tensor_by_name('input:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')
    
    #Multiply by batch_size to match the model's input parameters
    answer_logits = sess.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(text)]*batch_size, 
                                      source_sequence_length: [len(text)]*batch_size})[0] 


pad = source_letter_to_int["<PAD>"] 

print('Original Text:', input_sentence)

print('\nSource')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([source_int_to_letter[i] for i in text])))

print('\nTarget')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format(" ".join([target_int_to_letter[i] for i in answer_logits if i != pad])))

In [ ]:
### OLD

### Decoding - Training
Create training logits using [`tf.contrib.seq2seq.simple_decoder_fn_train()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_train) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder).  Apply the `output_fn` to the [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder) outputs.

### Decoding - Inference
Create inference logits using [`tf.contrib.seq2seq.simple_decoder_fn_inference()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/simple_decoder_fn_inference) and [`tf.contrib.seq2seq.dynamic_rnn_decoder()`](https://www.tensorflow.org/versions/r1.0/api_docs/python/tf/contrib/seq2seq/dynamic_rnn_decoder). 

### Build the Decoding Layer
Implement `decoding_layer()` to create a Decoder RNN layer.

- Create RNN cell for decoding using `rnn_size` and `num_layers`.
- Create the output fuction using [`lambda`](https://docs.python.org/3/tutorial/controlflow.html#lambda-expressions) to transform it's input, logits, to class logits.
- Use the your `decoding_layer_train(encoder_state, dec_cell, dec_embed_input, sequence_length, decoding_scope, output_fn, keep_prob)` function to get the training logits.
- Use your `decoding_layer_infer(encoder_state, dec_cell, dec_embeddings, start_of_sequence_id, end_of_sequence_id, maximum_length, vocab_size, decoding_scope, output_fn, keep_prob)` function to get the inference logits.

Note: You'll need to use [tf.variable_scope](https://www.tensorflow.org/api_docs/python/tf/variable_scope) to share variables between training and inference.